In [2]:
import pandas as pd
import numpy as np

# Survey

In [2]:
survey_raw = pd.ExcelFile('./Survey_RS_v5.xlsx')

In [3]:
survey_raw.sheet_names

['АНКЕТЫ_2018',
 'samples',
 'upload_b1',
 'upload_b2',
 'uploaded_ids',
 'relatives_m',
 'm1',
 'm2',
 'relatives_f',
 'f1',
 'f2',
 'regions',
 'populations',
 'population_gr',
 'profession',
 'education',
 'ethnogr',
 'eye_colors',
 'hair_color',
 'appearance',
 'Legend']

In [4]:
survey_raw.parse('АНКЕТЫ_2018')

,ДНК-происхождение-Бел,Unnamed: 1,Unnamed: 2,877,ДНК-популяция-Бел,Unnamed: 5,Unnamed: 6,Unnamed: 7,1078,ДНК-мегаполис-Бел,...,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71
0,№ п/п,Внутрилаб. №,Пол,Год рожд,Национальность,NaN,NaN,NaN,Дата отбора,место проживания,...,Другая информация,NaN,Фенотип,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,ДНК-происхождение-Бел,ДНК-популяция-Бел,ДНК-мегаполис-Бел,NaN,Страна,...,Образование,Род занятий,Цвет волос,Цвет глаз,Тип внешности,NaN,NaN,NaN,NaN,NaN
2,1,7217/0001,2,1983,Белорусы,NaN,NaN,+,2018-05-10 00:00:00,Беларусь,...,высшее,научный работник,темно-русый,карий,европеоидный,NaN,NaN,NaN,NaN,NaN
3,2,5212/0002,2,1983,Белорусы,NaN,NaN,+,2018-05-10 00:00:00,Беларусь,...,высшее,научный работник,темно-русый,серый,европеоидный,NaN,NaN,NaN,NaN,NaN
4,3,7216/0003,2,1966,Белорусы,NaN,NaN,+,2018-05-10 00:00:00,Беларусь,...,высшее,научный работник,темно-русый,серый,европеоидный,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2656,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2657,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2658,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2659,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# BelPOP data 2018

In [2]:
belpop_raw = pd.ExcelFile('./Bel_pop_data_18_loci.xls')

In [3]:
belpop_raw.sheet_names

['Полоцк',
 'Пинск',
 'Молодечно',
 'Мядель',
 'Крупки',
 'Сморгонь',
 'Иваново',
 'Городок',
 'Климовичи',
 'Светлогорск',
 'Барановичи',
 'Регионы']

In [4]:
regions = belpop_raw.parse('Регионы')
regions.columns = ['region', 'populations']
regions['populations'] = regions['populations'].apply(lambda x: x.split(', '))
regions = regions.explode('populations').reset_index(drop=True)

In [5]:
bel_pop_df = []
for sheet_name in belpop_raw.sheet_names[:-1]:
    sample = belpop_raw.parse(sheet_name, skiprows=1)
    loci = sample.iloc[0][2::2].index.tolist()
    sample_df = pd.DataFrame(sample.values[:, 2:],
                             columns=[f'{x}-{i % 2}' for i, x in enumerate(np.repeat(loci, 2))])
    sample_df['population'] = sheet_name
    sample_df['id'] = sample.values[:, 0]
    bel_pop_df.append(sample_df)
bel_pop_df = pd.concat(bel_pop_df).reset_index(drop=True)

In [6]:
bel_pop_df_final = bel_pop_df.merge(
    regions,
    left_on='population', right_on='populations'
).drop('populations', axis=1)

In [8]:
bel_pop_df_final.to_csv('bel_pop_2018_copy.csv', index=False)

In [3]:
bel_pop_df_final = pd.read_csv('bel_pop_2018_copy.csv')
bel_pop_df_final.columns = [x.replace(' ', '').replace('_', '') for x in bel_pop_df_final.columns]
bel_pop_df_final.to_csv('bel_pop_2018_copy.csv', index=False)

In [11]:
regions.to_csv('bel_pop_2018_regions.csv', index=False)

# Latest autosomal data

In [12]:
auto_raw = pd.ExcelFile('./autosomal_data-2020-03-04.xlsx')

In [13]:
auto_raw.sheet_names

['ТОТАЛ_Fusion', 'ТОТАЛ_ID6Plex', 'Лист3', 'БАЗА_АУТОСОМ_2019', 'Лист1']

In [34]:
sample = auto_raw.parse('БАЗА_АУТОСОМ_2019', skiprows=0)
display(sample.head())
loci = sample.iloc[0][3::2].index.tolist()
sample_df = pd.DataFrame(sample.values[:, 3:],
                            columns=[f'{x.strip()}-{i % 2}' for i, x in enumerate(np.repeat(loci, 2))])
sample_df['region'] = sample.values[:, 2]
sample_df['nationality'] = sample.values[:, 1]
sample_df['id'] = sample.values[:, 0]
sample_df.to_csv('autosomal_2019.csv', index=False)
sample_df

,№ п/п,Национальность,Этно-историч. регион,D3S1358,Unnamed: 4,D1S1656,Unnamed: 6,D2S441,Unnamed: 8,D10S1248,...,D9S1122,Unnamed: 50,D20S482,Unnamed: 52,D6S1043,Unnamed: 54,D17S1301,Unnamed: 56,D4S2408,Unnamed: 58
0,17,Русские,Центральный,14,15,16.0,17.3,10.0,14.0,13,...,12,13,13.0,15,11,11,11,13,9,11
1,39,Белорусы,Поднепровье,15,17,14.0,17.3,10.0,14.0,14,...,11,12,14.0,15,11,16,11,11,9,10
2,46,Белорусы,Центральный,17,18,12.0,17.3,11.0,15.0,14,...,12,13,16.0,16,11,18,12,12,9,9
3,85,Белорусы,Центральный,15,15,18.3,18.3,11.0,11.0,14,...,12,12,14.0,15,10,12,11,13,8,9
4,103,Белорусы,Центральный,18,18,11.0,12.0,11.0,14.0,13,...,12,12,14.0,16,11,11,10,13,8,11


,D3S1358-0,D3S1358-1,D1S1656-0,D1S1656-1,D2S441-0,D2S441-1,D10S1248-0,D10S1248-1,D13S317-0,D13S317-1,...,D20S482-1,D6S1043-0,D6S1043-1,D17S1301-0,D17S1301-1,D4S2408-0,D4S2408-1,region,nationality,id
0,14,15,16,17.3,10,14,13,16,8,10,...,15,11,11,11,13,9,11,Центральный,Русские,17
1,15,17,14,17.3,10,14,14,15,8,12,...,15,11,16,11,11,9,10,Поднепровье,Белорусы,39
2,17,18,12,17.3,11,15,14,15,11,11,...,16,11,18,12,12,9,9,Центральный,Белорусы,46
3,15,15,18.3,18.3,11,11,14,15,9,11,...,15,10,12,11,13,8,9,Центральный,Белорусы,85
4,18,18,11,12,11,14,13,14,8,11,...,16,11,11,10,13,8,11,Центральный,Белорусы,103
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
543,15,17,14,17.3,11,14,13,13,11,12,...,14,11,15,10,12,8,9,Поозерье,Белорусы,89s
544,17,17,15.3,17.3,11.3,14,13,14,11,12,...,13,12,14,12,12,8,9,Поозерье,Белорусы,8s
545,16,16,16,17.3,11,14,13,16,12,12,...,14,11,19,12,13,8,10,Поозерье,Белорусы,90s
546,14,17,13,15,14,15,14,15,10,12,...,15,14,16,11,12,8,11,Поднепровье,Белорусы,98kl


In [5]:
bel_pop_df_final = pd.read_csv('1036_dataset.csv')
bel_pop_df_final.columns = [x.replace(' ', '').replace('_', '') for x in bel_pop_df_final.columns]
bel_pop_df_final.to_csv('1036_dataset.csv', index=False)

# 1036

In [36]:
_1036_raw = pd.ExcelFile('./1036-Revised-Genotypes-PopStats-July-19-2017.xlsx')

In [37]:
_1036_raw.sheet_names

['1036 revised all, n=1036',
 '1036 revised AfAm, n=342',
 '1036 revised Cauc, n= 361',
 '1036 revised Hisp, n=236',
 '1036 revised Asian, n=97']

In [38]:
sample = _1036_raw.parse('1036 revised all, n=1036')

In [44]:
display(sample.head())
loci = sample.iloc[0][2::2].index.tolist()
sample_df = pd.DataFrame(sample.values[:, 2:],
                         columns=[f'{x.strip()}-{i % 2}' for i, x in enumerate(np.repeat(loci, 2))])
sample_df.head()
# sample_df['region'] = sample.values[:, 2]
sample_df['population'] = sample.values[:, 1]
sample_df['id'] = sample.values[:, 0]
sample_df.to_csv('1036_dataset.csv', index=False)
# sample_df

,Sample,Pop,CSF1PO,Unnamed: 3,D10S1248,Unnamed: 5,D12S391,Unnamed: 7,D13S317,Unnamed: 9,...,Penta_E,Unnamed: 51,SE33,Unnamed: 53,TH01,Unnamed: 55,TPOX,Unnamed: 57,vWA,Unnamed: 59
0,GT37019,AA,11,12,14,14,17.0,21.0,11,12,...,5.0,13.0,14.0,23.2,6.0,7.0,8.0,9.0,17,17
1,GT37020,AA,9,11,12,13,16.0,16.0,11,13,...,5.0,8.0,14.0,20.0,7.0,9.0,8.0,8.0,16,16
2,GT37026,AA,10,12,14,15,16.0,20.0,12,12,...,12.0,14.0,17.0,22.2,7.0,9.3,8.0,8.0,16,17
3,GT37027,AA,11,12,11,14,16.0,19.0,12,13,...,14.0,16.0,19.0,20.0,7.0,7.0,8.0,8.0,17,18
4,GT37032,AA,8,12,14,14,15.0,18.0,11,12,...,5.0,7.0,14.0,20.0,7.0,8.0,8.0,9.0,15,16
